In [5]:
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR10",
            # 'dataset': "MNIST",
            'fairness_agg': 'mean',
            'method': 'joint',
            'random_class_idx': False,

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 15,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 0,
            
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.01,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:5' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.00,
            'lambda': .0,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=CIFAR10/seed=10_epoch=15_lr=0.01_alpha=0.0_tau=0


In [6]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import BiasedMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                    per_task_memory_examples=params['per_task_memory_examples'],
                    per_task_examples = params['per_task_examples'],
                    joint = True,
                    random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = (params['method'] == "joint"),
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = True,

                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                joint = True,
                                random_class_idx=False)
    input_dim = (3, 28, 28)

else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)



joint=True
[0 1 2 3 4 5 6 7 8 9]


Files already downloaded and verified
Files already downloaded and verified
train_task=[1]
train_task=[1, 2]
train_task=[1, 2, 3]
train_task=[1, 2, 3, 4]
train_task=[1, 2, 3, 4, 5]


In [7]:
np.unique(benchmark.trains[4].targets)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [4]:
from algorithms.imbalance import Heuristic2
from algorithms import Heuristic3
from metrics import MetricCollector2, FairMetricCollector
from backbones import MLP2Layers2

backbone = MLP2Layers2(
    input_dim=input_dim, 
    hidden_dim_1=256, 
    hidden_dim_2=256, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [5]:
metric_manager_callback.meters['accuracy'].data.shape

(6, 6, 15)

In [6]:
# from trainers import ContinualTrainer
from trainers.fair_trainer import FairContinualTrainer2 as ContinualTrainer
# from trainers.imbalance_trainer import ImbalanceContinualTrainer1 as ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
if params['fairness_agg'] == "mean":
    agg = np.mean
elif params['fairness_agg'] == "max":
    agg = np.max
else:
    raise NotImplementedError

fairness_metrics = ["std", "EER", "EO", "DP"]
for metric in metric_manager_callback.meters:
    if metric in fairness_metrics:
        metric_manager_callback.meters[metric].agg = agg


In [8]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.9751393508945428, 'loss': 0.00020905635352675797, 'EO': [0.10444444444444445, 0.003669724770642202], 'DP': None, 'accuracy_s0': 1.0, 'accuracy_s1': 0.9459429153924567, 'classwise_accuracy': {0: array([933, 980]), 1: array([1133, 1135])}}
[2] Eval metrics for task 1 >> {'accuracy': 0.9920704845814978, 'loss': 0.00014037136120576384, 'EO': [0.03302752293577982, 0.0], 'DP': None, 'accuracy_s0': 1.0, 'accuracy_s1': 0.9834862385321101, 'classwise_accuracy': {1: array([1117, 1135]), 0: array([980, 980])}}
[3] Eval metrics for task 1 >> {'accuracy': 0.998608738649645, 'loss': 5.0619832211422866e-05, 'EO': [0.0022222222222222365, 0.003669724770642202], 'DP': None, 'accuracy_s0': 1.0, 'accuracy_s1': 0.9970540265035678, 'classwise_accuracy': {0: array([979, 980]), 1: array([1133, 1135])}}
[4] Eval metrics for task 1 >> {'accuracy': 0.9972871527465612, 'loss': 4.5737390126812826e-05, 'EO': [0

In [9]:
metric_manager_callback.meters['EER'].get_eer()

KeyError: 'EER'

In [ ]:
metric_manager_callback.meters['std'].get_std()

In [ ]:
metric_manager_callback.meters['EER'].get_eer()

In [ ]:
metric_manager_callback.meters['std'].get_std()

In [ ]:
metric_manager_callback.meters['accuracy'].get_data()

In [ ]:
metric_manager_callback.meters['accuracy'].compute_overall()

In [ ]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"EER:{np.mean(metric_manager_callback.meters['EER'].compute_overall())}")
print(f"std:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")
